In [1]:
import random
from collections import Counter

DECK_COMPLETE = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A',
                 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A',
                 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A',
                 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A']

#hit on 8 or less
#stay on 17 or more
#line determined by player's hand score, starting on a 9
#row determined by dealer's hand score, starting on a 2
#H for hit, D for double, S for stand
BASIC_STRATEGY = [['H','D','D','D','D','H','H','H','H','H'],
                  ['D','D','D','D','D','D','D','D','H','H'],
                  ['D','D','D','D','D','D','D','D','D','H'],
                  ['H','H','S','S','S','H','H','H','H','H'],
                  ['S','S','S','S','S','H','H','H','H','H'],
                  ['S','S','S','S','S','H','H','H','H','H'],
                  ['S','S','S','S','S','H','H','H','H','H'],
                  ['S','S','S','S','S','H','H','H','H','H']]

In [2]:
def rand_card(deck_func):
    card = random.choice(deck_func)
    deck_func.remove(card)
    return card


def score(cards):
    score = 0
    possible_hands = []
    aces = 0

    for card in cards:
        if type(card) == int:
            score += card
        elif card in ["J", "Q", "K"]:
            score += 10
        else:
            score += 1
            aces += 1

    for i in range(aces + 1):
        if (score + i * 10) <= 21:
            possible_hands.append(score + i * 10)

    if (possible_hands) == []:
        return -1
    return max(possible_hands)

In [3]:
def bj_round(hit_on_17=False):

    deck_current = DECK_COMPLETE.copy()

    player_cards = [
        rand_card(deck_func=deck_current),
        rand_card(deck_func=deck_current),
    ]
    player_score = score(player_cards)
    dealer_cards = [rand_card(deck_func=deck_current)]
    dealer_score = score(dealer_cards)

    if player_score == 21:
        return ("Win", 21)

    game_state = "Player_Phase"
    while game_state == "Player_Phase":
        if player_score <= 8:
            strategy = "H"
        elif player_score >= 17 + hit_on_17:
            strategy = "S"
        else:
            strategy = BASIC_STRATEGY[(player_score - 9)][(dealer_score - 2)]

        if strategy == "S":
            game_state = "Dealer_Phase"
        else:
            player_cards.append(rand_card(deck_func=deck_current))
            player_score = score(player_cards)
            if player_score == -1:
                return ("Loss", player_score)
            elif strategy == "D":
                game_state = "Dealer_Phase"

    while dealer_score < 17 + hit_on_17:
        dealer_cards.append(rand_card(deck_func=deck_current))
        dealer_score = score(dealer_cards)
        if dealer_score == -1:
            if(strategy == "D"):
                return ("Win_Double", player_score)
            return ("Win", player_score)

    if player_score > dealer_score:
        if(strategy == "D"):
            return ("Win_Double", player_score)
        return ("Win", player_score)
    elif player_score == dealer_score:
        return ("Draw", player_score)
    elif player_score < dealer_score:
        return ("Loss", player_score)

In [4]:
bj_round()

('Loss', 17)

In [5]:
results = {"Win": [], "Win_Double": [], "Draw": [], "Loss": []}

for _ in range(10000000):
    round_result, round_score = bj_round()
    results[round_result].append(round_score)

for key, values in results.items():
    total = dict(Counter(values))
    print(f"{key}: {round(len(values)*100/10000000, 3)}% of all games")

    sorted_dict = dict(sorted(total.items(), reverse=True))

    for key, value in sorted_dict.items():
        print(f"{key}: {round(value*100/sum(sorted_dict.values()), 2)}%")
    print("")

Win: 38.177% of all games
21: 20.47%
20: 25.74%
19: 15.15%
18: 12.06%
17: 8.29%
16: 3.61%
15: 3.88%
14: 3.97%
13: 4.26%
12: 2.57%

Win_Double: 5.154% of all games
21: 33.13%
20: 21.18%
19: 12.17%
18: 6.45%
17: 4.42%
16: 4.4%
15: 4.34%
14: 4.32%
13: 4.27%
12: 4.59%
11: 0.72%

Draw: 8.551% of all games
21: 8.03%
20: 32.87%
19: 18.28%
18: 19.33%
17: 21.49%

Loss: 48.118% of all games
20: 4.02%
19: 7.49%
18: 11.15%
17: 15.5%
16: 5.33%
15: 5.61%
14: 5.68%
13: 5.99%
12: 3.77%
11: 0.1%
-1: 35.35%



In [6]:
def bj_round_till_win_single(cum_loss=0, penalty_per_loss=2):
    current_round = bj_round()
    if current_round[0] == "Win":
        return (current_round[0], current_round[1] - cum_loss)
    if current_round[0] == "Win_Double":
        return (current_round[0], current_round[1] - cum_loss + 2)
    if current_round[0] == "Draw":
        return bj_round_till_win_single(cum_loss, penalty_per_loss)
    if current_round[0] == "Loss":
        return bj_round_till_win_single(cum_loss + penalty_per_loss, penalty_per_loss)


bj_round_till_win_single()

('Win', 17)

In [36]:
# total -3, min score 7
for _ in range(4):
    player = []
    for _ in range(6):
        player.append(max(bj_round_till_win_single()[1] - 3, 7))
    print(sorted(player, reverse=True))

[14, 13, 12, 12, 11, 7]
[18, 15, 14, 13, 9, 7]
[17, 15, 14, 14, 13, 7]
[17, 16, 15, 14, 12, 11]


In [33]:
def bj_round_till_win_group(size=6, cum_loss=0, penalty_per_loss=1):
    group = []
    while len(group) < size:
        current_round = bj_round()
        if current_round[0] == "Win":
            group.append(current_round[1] - cum_loss)
        if current_round[0] == "Win_Double":
            group.append(current_round[1] - cum_loss + 2)
        if current_round[0] == "Loss":
            cum_loss += penalty_per_loss

    return sorted(group, reverse=True)


bj_round_till_win_group()

[17, 16, 13, 11, 7, 4]

In [34]:
for _ in range(4):
    print(bj_round_till_win_group())

[20, 17, 14, 11, 10, 9]
[20, 14, 12, 11, 9, 7]
[17, 16, 14, 14, 10, 9]
[20, 20, 18, 17, 11, 11]
